In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os
from sqlalchemy import create_engine

In [3]:
os.chdir("..")
print(os.getcwd())

c:\Users\valen\Desktop\etl_workshop002


In [ ]:
with open ("credentials.json", "r", encoding="utf-8") as file:
    credentials = json.load(file)

db_host = credentials["db_host"]
db_name = credentials["db_name"]
db_user = credentials["db_user"]
db_password = credentials["db_password"]

conn = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:5432/{db_name}?client_encoding=utf8")


In [5]:
query = "SELECT * FROM grammys_raw_data;"
df = pd.read_sql(query, conn)

df.head()

,index,year,title,published_at,updated_at,category,nominee,artist,workers,img,winner
0,0,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,Bad Guy,Billie Eilish,"Finneas O'Connell, producer; Rob Kinelski & Fi...",https://www.grammy.com/sites/com/files/styles/...,True
1,1,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,"Hey, Ma",Bon Iver,"BJ Burton, Brad Cook, Chris Messina & Justin V...",https://www.grammy.com/sites/com/files/styles/...,True
2,2,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,7 rings,Ariana Grande,"Charles Anderson, Tommy Brown, Michael Foster ...",https://www.grammy.com/sites/com/files/styles/...,True
3,3,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,Hard Place,H.E.R.,"Rodney “Darkchild” Jerkins, producer; Joseph H...",https://www.grammy.com/sites/com/files/styles/...,True
4,4,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,Talk,Khalid,"Disclosure & Denis Kosiak, producers; Ingmar C...",https://www.grammy.com/sites/com/files/styles/...,True


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4810 entries, 0 to 4809
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   index         4810 non-null   int64 
 1   year          4810 non-null   int64 
 2   title         4810 non-null   object
 3   published_at  4810 non-null   object
 4   updated_at    4810 non-null   object
 5   category      4810 non-null   object
 6   nominee       4804 non-null   object
 7   artist        2970 non-null   object
 8   workers       2620 non-null   object
 9   img           3443 non-null   object
 10  winner        4810 non-null   bool  
dtypes: bool(1), int64(2), object(8)
memory usage: 380.6+ KB


In [7]:
df.describe()

,index,year
count,4810.000000,4810.000000
mean,2404.500000,1995.566944
std,1388.671727,17.149720
min,0.000000,1958.000000
25%,1202.250000,1983.000000
50%,2404.500000,1998.000000
75%,3606.750000,2010.000000
max,4809.000000,2019.000000


In [8]:
df.describe(include="object")

,title,published_at,updated_at,category,nominee,artist,workers,img
count,4810,4810,4810,4810,4804,2970,2620,3443
unique,62,4,10,638,4131,1658,2366,1463
top,62nd Annual GRAMMY Awards (2019),2017-11-28T00:03:45-08:00,2019-09-10T01:08:19-07:00,Song Of The Year,Robert Woods,(Various Artists),"John Williams, composer (John Williams)",https://www.grammy.com/sites/com/files/styles/...
freq,433,4205,778,70,7,66,20,26


In [9]:
print(f"Number of duplicates: {df.duplicated().sum()}")

Number of duplicates: 0


In [10]:
print(f"Dataset Shape: {df.shape}")
print("\nMissing Values per Column:")
print(df.isnull().sum().sort_values())

Dataset Shape: (4810, 11)

Missing Values per Column:
index              0
year               0
title              0
published_at       0
updated_at         0
category           0
winner             0
nominee            6
img             1367
artist          1840
workers         2190
dtype: int64


In [11]:
cols = ['nominee', 'img', 'artist', 'workers']
coverage = df[cols].notna().mean().sort_values(ascending=True) * 100

print("Data coverage by columns with null values (% of present values):")
print(coverage)

Data coverage by columns with null values (% of present values):
workers    54.469854
artist     61.746362
img        71.580042
nominee    99.875260
dtype: float64


In [12]:
df.drop(columns=['index', 'published_at', 'updated_at', 'img'], inplace=True)

In [13]:
df[df["nominee"].isnull()]

,year,title,category,nominee,artist,workers,winner
2282,2000,43rd Annual GRAMMY Awards (2000),"Remixer of the Year, Non-Classical",None,None,None,True
2377,1999,42nd Annual GRAMMY Awards (1999),"Remixer Of The Year, Non-Classical",None,None,None,True
2473,1998,41st Annual GRAMMY Awards (1998),"Remixer Of The Year, Non-classical",None,None,None,True
2569,1997,40th Annual GRAMMY Awards (1997),"Remixer Of The Year, Non-Classical",None,None,None,True
4526,1965,8th Annual GRAMMY Awards (1965),Best New Country & Western Artist,None,None,None,True
4574,1964,7th Annual GRAMMY Awards (1964),Best New Country & Western Artist Of 1964,None,None,None,True
